In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
for dataset in ['Anemia'] :
    %run train_and_run_experiments_bc.py --dataset {dataset} --data_dir . --output_dir outputs_dev/ --attention all --encoder all

INFO - 2019-04-25 12:52:19,601 - instantiating class <class 'Transparency.model.modules.Encoder.Encoder'> from params {'vocab_size': 16274, 'embed_size': 300, 'kernel_sizes': (1, 3, 5, 7), 'hidden_size': 32, 'activation': 'relu', 'type': 'cnn', 'pre_embed': array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.01662421,  0.15650244, -1.34739511, ..., -0.9011226 ,
        -0.48693601,  0.23673869],
       [ 0.15909452, -0.58059606,  0.94677802, ..., -0.01344293,
         2.20132898,  1.44688395],
       ...,
       [-0.29259351, -0.1140397 , -0.05117514, ..., -0.23458527,
         0.38887212,  0.02516322],
       [-2.31787848, -2.58242869,  3.71838903, ..., -2.7987864 ,
        -2.65423322, -1.66355622],
       [-0.86957598, -0.08357183,  0.73072731, ..., -0.43616131,
         0.08505905, -0.51678509]])} and extras set()
INFO - 2019-04-25 12:52:19,602 - type = cnn
INFO - 2019-04-25 12:52:19,602 - instantiating class <class 'Transp

Setting Embedding



100%|██████████| 24/24 [00:00<00:00, 37.18it/s]

{'accuracy': 0.916, 'roc_auc': 0.9671604938271605, 'pr_auc': 0.9803523357292854}


,0,1,micro avg,macro avg,weighted avg
f1-score,0.886,0.933,0.916,0.910,0.916
precision,0.863,0.948,0.916,0.906,0.918
recall,0.911,0.919,0.916,0.915,0.916
support,270.000,480.000,750.000,750.000,750.000


...

,0,1,micro avg,macro avg,weighted avg
f1-score,0.847,0.901,0.88,0.874,0.881
precision,0.788,0.946,0.88,0.867,0.889
recall,0.915,0.859,0.88,0.887,0.880
support,460.000,802.000,1262.00,1262.000,1262.000


INFO - 2019-04-25 17:15:39,025 - instantiating class <class 'Transparency.model.modules.Encoder.Encoder'> from params {'vocab_size': 16274, 'embed_size': 300, 'projection': True, 'hidden_size': 128, 'activation': 'tanh', 'type': 'average', 'pre_embed': None} and extras set()
INFO - 2019-04-25 17:15:39,025 - type = average
INFO - 2019-04-25 17:15:39,025 - instantiating class <class 'Transparency.model.modules.Encoder.EncoderAverage'> from params {'vocab_size': 16274, 'embed_size': 300, 'projection': True, 'hidden_size': 128, 'activation': 'tanh', 'pre_embed': None} and extras set()
INFO - 2019-04-25 17:15:39,026 - vocab_size = 16274
INFO - 2019-04-25 17:15:39,026 - embed_size = 300
INFO - 2019-04-25 17:15:39,026 - projection = True
INFO - 2019-04-25 17:15:39,027 - hidden_size = 128
INFO - 2019-04-25 17:15:39,027 - activation = tanh
INFO - 2019-04-25 17:15:39,027 - pre_embed = None
INFO - 2019-04-25 17:15:39,054 - instantiating class <class 'Transparency.model.modules.Decoder.AttnDecoder

{'accuracy': 0.8795562599049128, 'roc_auc': 0.9560853301528787, 'pr_auc': 0.9731096151843859}


,0,1,micro avg,macro avg,weighted avg
f1-score,0.847,0.901,0.88,0.874,0.881
precision,0.788,0.946,0.88,0.867,0.889
recall,0.915,0.859,0.88,0.887,0.880
support,460.000,802.000,1262.00,1262.000,1262.000


100%|██████████| 40/40 [14:46<00:00, 43.32s/it]


In [ ]:
from Trainers.DatasetBC import *
from ExperimentsBC import *

In [ ]:
from Trainers.DatasetQA import *
from ExperimentsQA import *

In [ ]:
for k in datasets :
    print(k)
    if k == 'sst' :
        dataset = datasets[k]()
        generate_graphs_on_encoders(dataset)
        print('+'*700)

In [ ]:
dataset = datasets_ehr['mortality']()

In [ ]:
from Trainers.PlottingBC import *

In [ ]:
evaluator = run_evaluator_on_latest_model(dataset, config='lstm')
logodds_results = pload(evaluator.model, 'logodds_attention')
emax_jds, emax_adv_attn, emax_ad_y = plot_attn_diff(dataset, dataset.test_data, logodds_results, 
                                                    save_name='logodds_subs', dirname=evaluator.model.dirname)

In [ ]:
n = 308
a = print_adversarial_example(dataset.vec.map2words(dataset.test_data.X[n]), dataset.test_data.attn_hat[n], emax_adv_attn[n], latex=True)
dataset.test_data.yt_hat[n], emax_ad_y[n]
print(a[2])

In [ ]:
ent = [np.max(x) for x in emax_adv_attn]
plt.scatter(ent, emax_jds)

In [ ]:
idx_1 = set(np.where(np.array(ent) > 0.6)[0]) & set(np.where(np.array(emax_jds) > 0.4)[0])
idx_1

In [ ]:
train_dataset(dataset, 'logodds_lstm_reg')

In [ ]:
run_logodds_experiment(dataset, 'logodds_lstm_reg')

In [ ]:
run_experiments_on_latest_model(dataset, 'logodds_lstm_reg')

In [ ]:
generate_graphs_on_latest_model(dataset, 'logodds_lstm_reg')

In [ ]:
run_logodds_substitution_experiment(dataset)

In [ ]:
test_data = dataset.test_data
plt.scatter(test_data.yt_hat[:, 0], test_data.opp_yt_hat[:, 0], s=5)
plt.show()

In [ ]:
diff = np.abs(test_data.yt_hat[:, 0] - test_data.opp_yt_hat[:, 0])
np.argsort(diff)[:15]

In [ ]:
plt.hist(diff, bins=30)

In [ ]:
test_data = dataset.test_data
for k, v in test_data.logodds_combined[0].items() :
    print(dataset.vec.map2words(v))

In [ ]:
n = 4231
true_X = dataset.vec.map2words(test_data.X[n])
new_X = dataset.vec.map2words(test_data.opp_X[n])
print_attn(true_X, dataset.test_data.attn_hat[n])
print(test_data.yt_hat[n])
print_attn(new_X, dataset.test_data.opp_attn[n])
print(test_data.opp_yt_hat[n])

In [ ]:
for k in datasets_ehr :
    dataset = datasets_ehr[k]()
    generate_graphs_on_latest_model(dataset)

In [ ]:
for k in datasets_ehr :
    dataset = datasets_ehr[k]()

In [ ]:
for k in datasets_ehr :
    dataset = datasets_ehr[k]()
    train_dataset(dataset, 'logodds_lstm')

In [ ]:
for k in datasets_ehr :
    dataset = datasets_ehr[k]()
    run_evaluator_on_latest_model(dataset, 'logodds_lstm')

In [ ]:
from model.LR import LR

In [ ]:
for k in datasets_ehr :
    if k != 'pheno' : continue
    dataset = datasets_ehr[k]()
    train_lr_on_dataset(dataset)

In [ ]:
for k in datasets_ehr :
    dataset = datasets_ehr[k]()
    push_all_models(dataset, dataset.keys_to_use)

In [ ]:
for k in datasets_ehr :
    dataset = datasets_ehr[k]()
    run_logodds_experiment(dataset)

In [ ]:
list(enumerate(dataset.vec.label_headers))

In [ ]:
get_top_words(dataset, config='lstm')

In [ ]:
y = np.array(dataset.test_data.yt_hat)
idx_y = np.where(y > 0.8)[0]

In [ ]:
from collections import defaultdict
top_words_dict = defaultdict(float)
for i in idx_y :
    d = dataset.test_data.top_words_attn[i]
    for k, v in d.items() :
        top_words_dict[k] += v

In [ ]:
top_words = dict(sorted(top_words_dict.items(), key=lambda x: x[1])[-20:])

In [ ]:
top_words

In [ ]:
lr.print_all_features(n=40)

In [ ]:
top_words_attn = set(top_words.keys())
top_words_lr = set(lr.get_features(n=20))

In [ ]:
top_words_attn & top_words_lr

In [ ]:
len(top_words_attn & top_words_lr) / len(top_words_attn | top_words_lr)

In [ ]:
dataset.keys_to_use

Multi Adversarial Examples
==========================

In [ ]:
for k in datasets_ehr :
    dataset = datasets_ehr[k]()
#     generate_adversarial_examples(dataset, config='lstm')
#     generate_logodds_examples(dataset, config='lstm')
#     generate_graphs_on_latest_model(dataset, config='lstm')
    push_all_models(dataset, keys=dataset.keys_to_use)

In [ ]:
dataset.vec.label_headers

In [ ]:
for k in datasets_ehr :
    dataset = datasets_ehr[k]()
    dataset.display_stats()

In [ ]:
len(dataset.test_data.X)

In [ ]:
evaluator = run_evaluator_on_latest_model(dataset)

In [ ]:
multi_adversarial_outputs = pload(evaluator.model, 'multi_adversarial')

In [ ]:
from Trainers.PlottingBC import *
test_data = dataset.test_data
emax_jds, emax_adv_attn, emax_ad_y = plot_multi_adversarial(test_data.X, test_data.yt_hat, 
                                                            test_data.attn_hat, multi_adversarial_outputs, dirname=".")

In [ ]:
print_adversarial_examples(dataset, test_data.X, test_data.yt_hat, test_data.attn_hat, 
                           emax_jds, emax_adv_attn, emax_ad_y, by_class=None, dirname='.')

In [ ]:
import numpy as np
a = np.random.randn(100, 10) + 100

In [ ]:
a.mean(0).min(), a.min(0).mean()